<a href="https://colab.research.google.com/github/Komal77rao/Data-Eng-Modules/blob/main/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class Functions Reading

In [ ]:


#git clone https://github.com/apis-jigsaw/class-functions-reading

### Introduction

In this lesson we'll learn about class methods.  Class methods methods that allow us to ask questions or perform operations on a *collection* of instances, as opposed to an individual instance.  We'll see that these methods often align with the SELECT commands in our relational databases.

### Sending Messages to Instances

So far, when we have written our classes we have relied on instance methods when coding out our functions.  For example, imagine we want to add a function called `is_expensive` to each `ingredient` instance.

In [ ]:
class Ingredient:
    def is_expensive(self):
        return self.price > 10

In [ ]:
salmon = Ingredient()
salmon.price = 12
salmon.is_expensive()

True

Notice that we can think of our function as allowing us to ask a question of the instance.  When we call `is_expensive`, we send a message to the instance and it responds in turn.  From inside the class, the receiver of the method call is `self`.

### Sending messages to classes

Now we can also ask questions of our classes.  What types of questions?  Well, remember we think of our class as a factory.  And so we might ask our ingredient factory questions about the collection of records.  For example, what is the `most_expensive` ingredients, a list of all expensive ingredients, etc.  In other words, any questions where we have one or more instances returned should use a class method.  

Ok, let's see how we can write a class method in Python.

In [ ]:
class Ingredient:
    def is_expensive(self):
        return self.price > 10

    @classmethod
    def most_expensive(self):
        print(self)

So we specify a class method with the `@classmethod` decorator.  

> For now, we are only printing out `self` in the body of the method.

Ok, let's see how we can call our function.

In [ ]:
Ingredient.most_expensive()

<class '__main__.Ingredient'>


Notice that we call a class method not on an instance, but on the class itself.  This makes sense: we are asking factory to answer a question.  Also, notice that when we print out `self` is not an instance but the `Ingredient` class.  

> This follows our old rule that `self` is whatever is to the left of the `.`.  Here, we have `Ingredient.most_expensive()`.

### Connecting Class Methods to the Database

If we think about class methods, like `Ingredient.most_expensive`, `Ingredient.expensive`, or `Ingredient.find_by_name`, we can see that these methods often translate to query methods in our SQL database.  For example here is how that `Ingredient.most_expensive` may look when it's filled in.

> Let's load some data.

In [ ]:
import sqlite3
conn = sqlite3.connect('./moes_bar.db')
cursor = conn.cursor()

import pandas as pd
root_url = "https://raw.githubusercontent.com/jigsawlabs-student/curriculum-images/main/has-many-through-bar/data/"
names = ['bartenders', 'customers', 'drinks', 'orders', 'ingredients', 'ingredients_drinks']
loaded_dfs = [pd.read_csv(f'{root_url}{name}.csv') for name in names]
for index, name in enumerate(names):
    loaded_dfs[index].to_sql(f'{name}', conn, index = False, if_exists = 'replace')
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('bartenders',),
 ('customers',),
 ('drinks',),
 ('orders',),
 ('ingredients',),
 ('ingredients_drinks',)]

In [ ]:
class Ingredient:
    __table__ = 'ingredients'
    columns = ['id', 'name', 'price']
    def is_expensive(self):
        return self.price > 10

    @classmethod
    def most_expensive(self, cursor):
        cursor.execute(f"SELECT * FROM {self.__table__} ORDER BY price DESC LIMIT 1")
        record = cursor.fetchone()
        return record

    @classmethod
    def expensive(self, cursor, price = 3):
        cursor.execute(f"SELECT * FROM ingredients WHERE price > ?", (price,))
        records = cursor.fetchall()
        return records

In [ ]:
Ingredient.expensive(cursor)

[(2, 'tonic', 4), (8, 'duff beer', 5)]

In [ ]:
Ingredient.most_expensive(cursor)

(8, 'duff beer', 5)

So notice that our class methods are called on the class itself, and that they relate to questions that are about the entire collection (eg. of all of the ingredients, which are expensive), and involve searching through the collection of ingredients to find an answer.

### Summary

In this lesson, we saw how to implement class methods and how they align with our SQL queries. Now we use class methods when we want to ask a question that pertains to a collection of instances.  

> Oftentimes, what's returned from our class method is an instance or a collection of instances.   

To implement a `classmethod` we use `@classmethod` decorator.  

In [ ]:
class Ingredient:
    @classmethod
    def most_expensive(self, cursor):
        cursor.execute(f"SELECT * FROM ingredients ORDER BY price DESC LIMIT 1")
        record = cursor.fetchone()
        return record

And we call a class method with the name of the class followed by the method name.

In [ ]:
Ingredient.most_expensive(cursor)

(8, 'duff beer', 5)

Notice that when calling a class method, self is our class itself (here Ingredient).  And this follows our rule that self is whatever object is to the left of the dot, in our method call.

These class methods align with our `where` select statements.  This is because, to answer a question about the collection, we need to perform a select statement to find the row or rows of data that contain the desired result.